In [1]:
# Load VGG16 and ResNet models
from tensorflow.keras.models import load_model
vgg16_model = load_model('vgg16.h5')
resnet_model = load_model('resnet.h5')
capsnet_model = load_model('capsnet.h5')

In [2]:
# re-size all the images to standard size 224 x 224
IMAGE_SIZE = [224, 224]

# specifying path for train and test data folders
train_path = 'C:/Ajay/documents/Major project/data/train'
valid_path = 'C:/Ajay/documents/Major project/data/test'

In [3]:
# Image Data Generator to import the images from the dataset
from keras.preprocessing.image import ImageDataGenerator
# Addition of data-augmentation parameters to ImageDataGenerator
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   width_shift_range = 0.2,
                                   height_shift_range = 0.2,
                                   rotation_range = 40,
                                   horizontal_flip = True)

In [4]:
# validation data should not be augmented
test_datagen = ImageDataGenerator(rescale = 1./255)

training_set = train_datagen.flow_from_directory('C:/Ajay/documents/Major project/data/train',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

test_set = test_datagen.flow_from_directory('C:/Ajay/documents/Major project/data/test',
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 2637 images belonging to 2 classes.
Found 660 images belonging to 2 classes.


In [5]:
X_test, y_test = test_set.next()

In [6]:
# Generate predictions on a validation set
vgg16_preds = vgg16_model.predict(X_test)

In [7]:
resnet_preds = resnet_model.predict(X_test)

In [8]:
capsnet_preds = capsnet_model.predict(X_test)

In [9]:
import numpy as np
import pandas as pd
# Combine the predictions using a majority voting scheme
ensemble_pred = np.round((vgg16_preds + resnet_preds + capsnet_preds) / 3)

In [10]:
# Evaluate the accuracy of the ensemble model
accuracy = np.mean(ensemble_pred == y_test)

In [11]:
print("Accuracy of the ensemble model:", accuracy)

Accuracy of the ensemble model: 0.90625


In [12]:
# Generate the classification report for the ensemble model
from sklearn.metrics import classification_report
report = classification_report(y_test, ensemble_pred)

print(report)

              precision    recall  f1-score   support

           0       1.00      0.82      0.90        17
           1       0.83      1.00      0.91        15

   micro avg       0.91      0.91      0.91        32
   macro avg       0.92      0.91      0.91        32
weighted avg       0.92      0.91      0.91        32
 samples avg       0.91      0.91      0.91        32



In [13]:
import numpy as np

np.save('ensemble_preds.npy', ensemble_pred)